In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from tensorflow.keras import  models, optimizers, layers, activations
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import History
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception
from tensorflow.keras.applications.resnet50 import ResNet50

In [2]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D ,GlobalAveragePooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import BatchNormalization

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7419851313319150085
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2258055988
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4853429039754009430
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [4]:
path = "E:\deep learning project\inaturalist_12K"

In [5]:
image_size=(256,256,3)
output_size=10

In [6]:
def get_data(path,augmentation=True):
    train_path=os.path.join(path,"train")
    test_path=os.path.join(path,"val")
    if augmentation==False:
        train_generator=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split=0.1)
    else:
        train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                          rotation_range=90,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          validation_split=0.1,
                                          horizontal_flip=True)
    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    train_data = train_generator.flow_from_directory(
    directory=train_path,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=33)
    valid_data=train_generator.flow_from_directory(
    directory=train_path,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=33)
    test_data=test_generator.flow_from_directory(
    directory=test_path,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=33)
    return train_data,valid_data,test_data

In [7]:
model_b=False

In [8]:
class inc_v2(object):
    def __init__(self,input_size,output_size,drop_out,batch_normalization,dense_size):
        self.input_size =input_size
        self.output_size=output_size
        self.drop_out=drop_out
        self.batch_normalization=batch_normalization
        self.initialize(input_size,output_size,drop_out,batch_normalization,dense_size)
    def initialize(self,input_size,output_size,drop_out,batch_normalization,dense_size):
        self.model_b=InceptionResNetV2(include_top=False,
            weights="imagenet",
            input_shape=input_size,
        )
        self.model_b.trainable=False
        input = keras.Input(shape=input_size)
        x = self.model_b(input, training=False)
        self.model=Sequential()
        self.model.add(Model(input, x))
        self.model.add(Flatten())
        self.model.add(Dense(dense_size))
        self.model.add(Activation('selu'))
        if batch_normalization==True:
            self.model.add(BatchNormalization())
        self.model.add(Dropout(drop_out))
        self.model.add(Dense(output_size))
        self.model.add(Activation("softmax"))



In [9]:
class inc_v3(object):
    def __init__(self,input_size,output_size,drop_out,batch_normalization,dense_size):
        self.input_size =input_size
        self.output_size=output_size
        self.drop_out=drop_out
        self.batch_normalization=batch_normalization
        self.initialize(input_size,output_size,drop_out,batch_normalization,dense_size)
    def initialize(self,input_size,output_size,drop_out,batch_normalization,dense_size):
        self.model_b=InceptionV3(include_top=False,
            weights="imagenet", 
            input_shape=input_size,
        )
        self.model_b.trainable=False
        input = keras.Input(shape=input_size)
        x = self.model_b(input, training=False)
        self.model=Sequential()
        self.model.add(Model(input, x))
        self.model.add(Flatten())
        self.model.add(Dense(dense_size))
        self.model.add(Activation('selu'))
        if batch_normalization==True:
            self.model.add(BatchNormalization())
        self.model.add(Dropout(drop_out))
        self.model.add(Dense(output_size))
        self.model.add(Activation("softmax"))

In [10]:
class Xcpt_ion(object):
    def __init__(self,input_size,output_size,drop_out,batch_normalization,dense_size):
        self.input_size =input_size
        self.output_size=output_size
        self.drop_out=drop_out
        self.batch_normalization=batch_normalization
        self.initialize(input_size,output_size,drop_out,batch_normalization,dense_size)
    def initialize(self,input_size,output_size,drop_out,batch_normalization,dense_size):
        self.model_b=Xception(include_top=False,
            weights="imagenet",
            input_shape=input_size)
        self.model_b.trainable=False
        input = keras.Input(shape=input_size)
        x = self.model_b(input, training=False)
        self.model=Sequential()
        self.model.add(Model(input, x))
        self.model.add(Flatten())
        self.model.add(Dense(dense_size))
        self.model.add(Activation('selu'))
        if batch_normalization==True:
            self.model.add(BatchNormalization())
        self.model.add(Dropout(drop_out))
        self.model.add(Dense(output_size))
        self.model.add(Activation("softmax"))

In [11]:
class Res_net(object):
    def __init__(self,input_size,output_size,drop_out,batch_normalization,dense_size):
        self.input_size =input_size
        self.output_size=output_size
        self.drop_out=drop_out
        self.batch_normalization=batch_normalization
        self.initialize(input_size,output_size,drop_out,batch_normalization,dense_size)
    def initialize(self,input_size,output_size,drop_out,batch_normalization,dense_size):
        self.model_b=ResNet50(include_top=False,
            weights="imagenet",
            input_shape=input_size)
        self.model_b.trainable=False
        input = keras.Input(shape=input_size)
        x = self.model_b(input, training=False)
        self.model=Sequential()
        self.model.add(Model(input, x))
        self.model.add(Flatten())
        self.model.add(Dense(dense_size))
        self.model.add(Activation('selu'))
        if batch_normalization==True:
            self.model.add(BatchNormalization())
        self.model.add(Dropout(drop_out))
        self.model.add(Dense(output_size))
        self.model.add(Activation("softmax"))

In [12]:
model=Res_net(image_size,output_size,0.4,True,32)
model.model.summary()

print(model.model_b.trainable)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 8, 8, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 32)                4194336   
_________________________________________________________________
activation (Activation)      (None, 32)                0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                3

In [13]:
trained_models={"inception_v2":inc_v2,"inception_v3":inc_v3,"Xception":Xcpt_ion,"Resnet_50":Res_net}

In [14]:
import wandb
wandb.login()

wandb: Currently logged in as: akashsainics21m003 (use `wandb login --relogin` to force relogin)


True

In [15]:
from wandb.keras import WandbCallback

# WITHOUT FINETUNING PRETRAINED MODELS 

In [17]:

import numpy as np
from tensorflow.keras.optimizers import Adam
np.random.seed(2)
class Runner_1(object):
    def __init__(self,inaturalist_data=True,augmentation=True):
        if inaturalist_data:
            self.initialize_inaturalist_data(augmentation)
        else:
            self.initialize_data()

    def initialize_inaturalist_data(self,augmentation=True):
        self.train_data,self.valid_data,self.test_data= get_data(path,augmentation)
    

    def initialize_data(self):
        raise NotImplementedError("Please implement this method if you need other dataset.")

    @staticmethod
    def get_activation_function(key):
        mapper = {
      #"sigmoid": Sigmoid,
      "relu": 'relu'
      #"tanh": Tanh
    }
        assert key in mapper
        return mapper[key]

def run_wandb_1():
    wandb.init(project="without_pretrained_model", entity="cs21m003_cs21d406")
    config = wandb.config
    base_model=trained_models[config.models]
    wandb.run.name=f"model_{config.models}"
    runner=Runner_1(True,True)
    """
    params = {
    "models": config.models

    }
    """
    model_1=base_model(image_size,output_size,0.5,True,64)
    model_1.model.compile(
    optimizer=Adam(0.0001),  # Optimizer
    loss="categorical_crossentropy", metrics="categorical_accuracy")
    model_1.model.fit(runner.train_data,epochs=5,batch_size=64,validation_data=runner.valid_data,
          callbacks=[WandbCallback()])
    loss,accuracy=model_1.model.evaluate(runner.test_data,batch_size=64)
    model_1.model_b.trainable=True
    print(model_1.model_b.trainable)
    print(f'test accuracy:{accuracy}')
    wandb.log({"test accuracy":accuracy})

def do_hyperparameter_search_using_wandb_1():
    sweep_config = {
    "name": "grid sweep",
    "method": "grid",
    "metric":{
      "name": "ValidationAccuracy",
      "goal": "maximize"
    },
    "parameters":{
      "models": {"values": ["inception_v3","Xception","Resnet_50"]}}}
  
    sweep_id = wandb.sweep(sweep_config, project = "without_pretrained_model",entity='cs21m003_cs21d406')
    wandb.agent(sweep_id, function=run_wandb_1,count=3)

In [18]:
do_hyperparameter_search_using_wandb_1()

Create sweep with ID: 43gzdvb5
Sweep URL: https://wandb.ai/cs21m003_cs21d406/without_pretrained_model/sweeps/43gzdvb5


wandb: Agent Starting Run: ceh5oznp with config:
wandb: 	models: inception_v3
wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Found 9999 images belonging to 10 classes.
Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/5
313/313 [==============================] - 753s 2s/step - loss: 1.3167 - categorical_accuracy: 0.6073 - val_loss: 0.9136 - val_categorical_accuracy: 0.7179
Epoch 2/5
313/313 [==============================] - 615s 2s/step - loss: 1.0732 - categorical_accuracy: 0.6644 - val_loss: 0.8562 - val_categorical_accuracy: 0.7227
Epoch 3/5
313/313 [==============================] - 572s 2s/step - loss: 0.9954 - categorical_accuracy: 0.6855 - val_loss: 0.7928 - val_categorical_accuracy: 0.7393
Epoch 4/5
313/313 [==============================] - 554s 2s/step - loss: 0.9340 - categorical_accuracy: 0.7065 - val_loss: 0.7839 - val_categorical_accuracy: 0.7484
Epoch 5/5
63/63 [==============================] - 48s 771ms/step - loss: 0.6465 - categorical_accuracy: 0.7925
True
test accuracy:0.7925000190734863



categorical_accuracy,▁▅▆██
epoch,▁▃▅▆█
loss,█▄▂▁▁
test accuracy,▁
val_categorical_accuracy,▁▂▄▅█
val_loss,█▆▄▄▁
best_epoch,4
best_val_loss,0.7071
categorical_accuracy,0.71007
epoch,4
loss,0.91287


wandb: Agent Starting Run: 0bz003eq with config:
wandb: 	models: Xception
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


...

Epoch 2/5
313/313 [==============================] - 631s 2s/step - loss: 1.1024 - categorical_accuracy: 0.6614 - val_loss: 0.8348 - val_categorical_accuracy: 0.7207
Epoch 3/5
313/313 [==============================] - 610s 2s/step - loss: 1.0353 - categorical_accuracy: 0.6785 - val_loss: 0.7808 - val_categorical_accuracy: 0.7396
Epoch 4/5
313/313 [==============================] - 590s 2s/step - loss: 0.9905 - categorical_accuracy: 0.6831 - val_loss: 0.7692 - val_categorical_accuracy: 0.7480
Epoch 5/5
63/63 [==============================] - 45s 709ms/step - loss: 0.6716 - categorical_accuracy: 0.7875
True
test accuracy:0.7875000238418579



categorical_accuracy,▁▆▇▇█
epoch,▁▃▅▆█
loss,█▄▃▂▁
test accuracy,▁
val_categorical_accuracy,▁▃▆▇█
val_loss,█▅▃▂▁
best_epoch,4
best_val_loss,0.73581
categorical_accuracy,0.69617
epoch,4
loss,0.94641


wandb: Agent Starting Run: g4lx50xz with config:
wandb: 	models: Resnet_50
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Found 9999 images belonging to 10 classes.
Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/5
313/313 [==============================] - 638s 2s/step - loss: 2.6057 - categorical_accuracy: 0.1583 - val_loss: 3.9441 - val_categorical_accuracy: 0.1014
Epoch 2/5
313/313 [==============================] - 644s 2s/step - loss: 2.4304 - categorical_accuracy: 0.1854 - val_loss: 2.7249 - val_categorical_accuracy: 0.1628
Epoch 3/5
313/313 [==============================] - 664s 2s/step - loss: 2.3645 - categorical_accuracy: 0.1993 - val_loss: 2.5550 - val_categorical_accuracy: 0.1441
Epoch 4/5
313/313 [==============================] - 649s 2s/step - loss: 2.3319 - categorical_accuracy: 0.2069 - val_loss: 2.4830 - val_categorical_accuracy: 0.1784
Epoch 5/5
63/63 [==============================] - 46s 735ms/step - loss: 2.6505 - categorical_accuracy: 0.1715
True
test accuracy:0.17149999737739563



categorical_accuracy,▁▅▆▇█
epoch,▁▃▅▆█
loss,█▄▃▂▁
test accuracy,▁
val_categorical_accuracy,▁▇▅█▇
val_loss,█▂▁▁▂
best_epoch,3
best_val_loss,2.48302
categorical_accuracy,0.21072
epoch,4
loss,2.28805
